### 1. Implementación de la clase DWayHeap

La clase **DWayHeap** es una implementación genérica de un heap d-ario (o d-way heap) que funciona como una cola de prioridades. A diferencia del heap binario tradicional, este heap permite definir un **factor de ramificación** (o _branching factor_) arbitrario (mínimo 2) que indica el número máximo de hijos por nodo. Esto permite experimentar con distintas estructuras de árbol para balancear los costos de inserción y extracción.

#### Estructura interna

- **Representación interna:**  
  Se utiliza una lista llamada `_pairs` para almacenar tuplas del tipo `(priority, element)`, donde cada elemento se asocia a una prioridad. Esta representación lineal corresponde a un árbol completo alineado a la izquierda.

- **Atributos clave:**  
  - `self.D`: Representa el factor de ramificación (número de hijos por nodo).  
  - `self._pairs`: Lista de tuplas que almacena los elementos junto con sus prioridades.

#### Métodos y algoritmos
- **Inicialización (`__init__`):**  
  - Permite construir el heap a partir de dos listas: una de elementos y otra de prioridades.  
  - Se valida que ambas listas tengan la misma longitud y que el factor de ramificación sea al menos 2.  
  - Si se proporcionan elementos, se llama al método `_heapify` para organizar la estructura respetando las invariantes del heap.

- **Métodos de utilidad:**
  - `__len__` y `__sizeof__`: Devuelven el número de elementos en el heap.
  - `is_empty`: Indica si el heap no contiene elementos.
  - `first_leaf_index`: Calcula el índice de la primera hoja basándose en la fórmula para árboles completos en una representación lineal.

- **Operaciones de mantenimiento de invariantes:**
  - **_bubble_up:**  
    Permite "elevar" un elemento insertado hacia la raíz si su prioridad es mayor que la de su padre. Se intercambian posiciones hasta que se restablece la propiedad del heap.
    
  - **_push_down:**  
    Después de una extracción, este método "empuja" el elemento de la raíz hacia abajo, intercambiándolo con el hijo de mayor prioridad, para que la raíz siempre contenga la mayor prioridad.
    
  - **_heapify:**  
    Construye el heap a partir de listas desordenadas de elementos y prioridades. Se procede de forma "bottom-up" aplicando `_push_down` a cada nodo interno, asegurando que se cumpla la invariante de que cada nodo tiene la máxima prioridad en su subárbol.

- **Operaciones de acceso:**
  - **peek:** Devuelve el elemento con la mayor prioridad sin eliminarlo; lanza un error si el heap está vacío.
  - **top:** Extrae y devuelve el elemento de mayor prioridad, reestructurando el heap tras la extracción.
  - **insert:** Inserta un nuevo par (elemento, prioridad) al final de la lista y luego utiliza `_bubble_up` para ajustar su posición y mantener la propiedad del heap.

- **Verificación de invariantes:**  
  El método `_validate` recorre los nodos internos (hasta la primera hoja) y comprueba que cada nodo tenga una prioridad mayor o igual que la de cada uno de sus hijos. Esto es crucial para confirmar la integridad del heap tras las operaciones de inserción y extracción.

- **Manejo de errores:**  
  Se lanzan excepciones `ValueError` en el constructor si las listas tienen longitudes inconsistentes o si el factor de ramificación es inválido. Asimismo, se utiliza `RuntimeError` en métodos como `peek` y `top` cuando se operan en un heap vacío.
- **Eficiencia:**  
  Las operaciones principales (insertar, extraer) mantienen complejidades logarítmicas en base al número de elementos y al factor de ramificación, haciendo que la estructura sea eficiente para una amplia gama de aplicaciones.


In [ ]:
from typing import Any, List, Optional, Tuple

class DWayHeap(object):
    def __init__(self, elements: List[Any] = [], priorities: List[float] = [], branching_factor: int = 2) -> None:
        """
        Constructor

        Parámetros:
            elements: Los elementos para inicializar el heap.
            priorities: Las prioridades de los elementos anteriores. Deben tener la misma longitud que `elements`.
            branching_factor: El número (máximo) de hijos por nodo en el heap. Debe ser al menos 2.
        """
        if len(elements) != len(priorities):
            raise ValueError(f'La longitud de la lista de elementos ({len(elements)})'
                             f' debe coincidir con la longitud de la lista de prioridades ({len(priorities)}).')
        if branching_factor < 2:
            raise ValueError(f'El factor de ramificación ({branching_factor}) debe ser mayor que 1.')
        self._pairs: List[Tuple[float, Any]] = []
        self.D = branching_factor

        if len(elements) > 0:
            self._heapify(elements, priorities)

    def __sizeof__(self) -> int:
        """
        Retorna el tamaño del heap.

        Retorna: El número de elementos en el heap.
        """
        return len(self)

    def __len__(self) -> int:
        """
        Retorna el tamaño del heap.

        Retorna: El número de elementos en el heap.
        """
        return len(self._pairs)

    def _validate(self) -> bool:
        """
        Verifica que se cumplan las tres invariantes del heap:
        1. Cada nodo tiene como máximo `D` hijos. (Garantizado por la construcción)
        2. El árbol del heap es completo y alineado a la izquierda. (También garantizado por la construcción)
        3. Cada nodo contiene la mayor prioridad en el subárbol con raíz en ese nodo.

        Retorna: True si se cumplen todas las invariantes del heap.
        """
        current_index = 0
        first_leaf = self.first_leaf_index()
        while current_index < first_leaf:
            current_priority: float = self._pairs[current_index][0]
            first_child = self._first_child_index(current_index)
            last_child_guard = min(first_child + self.D, len(self))
            for child_index in range(first_child, last_child_guard):
                if current_priority < self._pairs[child_index][0]:
                    return False
            current_index += 1
        return True

    def _push_down(self, index: int) -> None:
        """
        Empuja hacia abajo la raíz de un sub-heap hacia sus hojas para restablecer las invariantes del heap.
        Si alguno de los hijos del elemento tiene mayor prioridad, se intercambia el elemento actual
        con su hijo de mayor prioridad, y se verifica recursivamente el sub-heap que estaba previamente
        enraizado en ese hijo.

        Parámetro:
            index: El índice de la raíz del sub-heap.
        """
        assert (0 <= index < len(self._pairs))
        input_pair = self._pairs[index]
        input_priority = input_pair[0]
        current_index = index
        first_leaf = self.first_leaf_index()
        while current_index < first_leaf:
            child_index = self._highest_priority_child_index(current_index)
            assert (child_index is not None)
            if self._pairs[child_index][0] > input_priority:
                self._pairs[current_index] = self._pairs[child_index]
                current_index = child_index
            else:
                break

        self._pairs[current_index] = input_pair

    def _bubble_up(self, index: int) -> None:
        """
        Eleva un elemento hacia la raíz para restablecer las invariantes del heap.
        Si el padre del elemento tiene menor prioridad, se intercambian el elemento actual y su padre,
        y se verifica recursivamente la posición anterior.

        Parámetro:
            index: El índice del elemento a elevar.
        """
        assert (0 <= index < len(self._pairs))
        input_pair = self._pairs[index]
        input_priority = input_pair[0]
        while index > 0:
            parent_index = self._parent_index(index)
            parent = self._pairs[parent_index]

            if input_priority > parent[0]:
                self._pairs[index] = parent
                index = parent_index
            else:
                break

        self._pairs[index] = input_pair

    def _first_child_index(self, index: int) -> int:
        """
        Calcula el índice del primer hijo de un nodo en el heap.

        Parámetro:
            index: El índice del nodo actual.

        Retorna: El índice del primer hijo (más a la izquierda) del nodo.
        """
        return index * self.D + 1

    def _parent_index(self, index: int) -> int:
        """
        Calcula el índice del padre de un nodo en el heap.

        Parámetro:
            index: El índice del nodo actual.

        Retorna: El índice del padre del nodo actual.
        """
        return (index - 1) // self.D

    def _highest_priority_child_index(self, index: int) -> Optional[int]:
        """
        Encuentra, entre los hijos de un nodo, el hijo con la mayor prioridad.
        Si varios hijos tienen la misma prioridad, se devuelve el más a la izquierda.

        Parámetro:
            index: El índice del nodo cuyos hijos se examinan.

        Retorna: El índice del hijo con mayor prioridad o None si no tiene hijos.
        """
        first_index = self._first_child_index(index)
        size = len(self)
        last_index = min(first_index + self.D, size)

        if first_index >= size:
            return None

        highest_priority = -float('inf')
        index_result = first_index
        for i in range(first_index, last_index):
            if self._pairs[i][0] > highest_priority:
                highest_priority = self._pairs[i][0]
                index_result = i

        return index_result

    def first_leaf_index(self) -> int:
        """
        Calcula el índice de la primera hoja en el heap.

        Retorna: El índice de la primera hoja.
        """
        return (len(self) - 2) // self.D + 1

    def _heapify(self, elements: List[Any], priorities: List[float]) -> None:
        """
        Inicializa el heap con una lista de elementos y prioridades.

        Parámetros:
            elements: La lista de elementos a agregar al heap.
            priorities: Las prioridades correspondientes a los elementos (en el mismo orden).
        """
        assert (len(elements) == len(priorities))
        self._pairs = list(zip(priorities, elements))
        last_inner_node_index = self.first_leaf_index() - 1
        for index in range(last_inner_node_index, -1, -1):
            self._push_down(index)

    def is_empty(self) -> bool:
        """
        Verifica si el heap está vacío.

        Retorna: True si el heap no tiene elementos.
        """
        return len(self) == 0

    def top(self) -> Any:
        """
        Elimina y devuelve el elemento con la mayor prioridad del heap.
        Si el heap está vacío, lanza un RuntimeError.

        Retorna: El elemento con la mayor prioridad.
        """
        if self.is_empty():
            raise RuntimeError('Se llamó al método top en un heap vacío.')
        if len(self) == 1:
            element = self._pairs.pop()[1]
        else:
            element = self._pairs[0][1]
            self._pairs[0] = self._pairs.pop()
            self._push_down(0)

        return element

    def peek(self) -> Any:
        """
        Devuelve, sin eliminar, el elemento con la mayor prioridad del heap.
        Si el heap está vacío, lanza un RuntimeError.

        Retorna: El elemento con la mayor prioridad.
        """
        if self.is_empty():
            raise RuntimeError('Se llamó al método peek en un heap vacío.')
        return self._pairs[0][1]

    def insert(self, element: Any, priority: float) -> None:
        """
        Agrega un nuevo par (elemento, prioridad) al heap.

        Parámetros:
            element: El elemento a agregar.
            priority: La prioridad asociada al elemento.
        """
        self._pairs.append((priority, element))
        self._bubble_up(len(self._pairs) - 1)


### 2. Pruebas unitarias con `unittest`

El bloque de pruebas está diseñado para verificar la correcta funcionalidad de la clase **DWayHeap**. Se emplea el framework `unittest` de Python para definir un conjunto de casos de prueba que aseguran tanto el correcto funcionamiento de la lógica del heap como la robustez ante condiciones límite y entradas inválidas.

#### Casos de prueba definidos

1. **Inicialización (`test_init`):**
   - **Heap vacío:** Se verifica que la creación de un heap sin elementos retorna una longitud cero.
   - **Validación de parámetros:**  
     Se comprueba que al pasar un factor de ramificación menor que 2 se lanza el error adecuado, verificando que el mensaje de error sea el esperado.
   - **Consistencia entre listas:**  
     Se prueba que si la longitud de la lista de elementos no coincide con la de prioridades, se arroja un `ValueError` con el mensaje correspondiente.
   - **Inicialización válida:**  
     Se crea un heap con listas de elementos y prioridades compatibles, verificando que la estructura cumpla las invariantes mediante el método `_validate`.

2. **Heapify (`test_heapify`):**
   - Se generan de forma aleatoria listas de elementos y prioridades para distintos factores de ramificación (2, 3, 4, 5, 6).
   - Se construye el heap mediante el método `_heapify` y se valida que la cantidad de elementos sea la correcta y que la estructura cumpla sus invariantes.

3. **Operaciones de inspección y vaciamiento (`test_clear`):**
   - Se prueba que el método `peek` lanza un error cuando se invoca en un heap vacío.
   - Se insertan elementos con distintas prioridades y se verifica que `peek` retorna correctamente el elemento de mayor prioridad.

4. **Inserción y extracción (`test_insert_top`):**
   - Se confirma que la extracción (`top`) en un heap vacío genera el error correspondiente.
   - Se verifica el correcto funcionamiento de la operación de inserción seguida de extracción, comprobando que la estructura se mantenga y que el tamaño se actualice de forma adecuada.
   - Se realizan inserciones adicionales de forma aleatoria para luego extraer elementos, validando en cada paso que el heap cumpla con las invariantes.

#### Importancia de las pruebas

- **Robustez y confiabilidad:**  
  Al cubrir casos de borde (por ejemplo, heap vacío, entradas inválidas) se asegura que la implementación es robusta.
- **Verificación de invariantes:**  
  La llamada frecuente al método `_validate` durante las extracciones confirma que cada operación mantiene la estructura correcta del heap.
- **Aleatoriedad:**  
  La generación de datos aleatorios en algunas pruebas refuerza la confianza en la estabilidad y desempeño de la implementación en condiciones variadas.


In [ ]:
import unittest
import random

# Factores de ramificación a probar
BRANCHING_FACTORS_TO_TEST = [2, 3, 4, 5, 6]

# Se asume que la clase DWayHeap ya está definida en el entorno.

class HeapTest(unittest.TestCase):
    def test_init(self):
        # Prueba de inicialización: se crea un heap vacío con factor de ramificación 2.
        heap = DWayHeap(branching_factor=2)
        self.assertEqual(0, len(heap))

        # Prueba con valores inválidos para el factor de ramificación (debe ser mayor que 1).
        for b in [1, 0, -1]:
            with self.assertRaises(ValueError) as context:
                DWayHeap(branching_factor=b)
            self.assertTrue(f'El factor de ramificación ({b}) debe ser mayor que 1.' in str(context.exception))

        # Prueba de inconsistencia entre la longitud de la lista de elementos y la lista de prioridades.
        with self.assertRaises(ValueError) as context:
            DWayHeap(priorities=[1.0])
        error_str = 'La longitud de la lista de elementos (0) debe coincidir con la longitud de la lista de prioridades (1).'
        self.assertTrue(error_str in str(context.exception))

        # Prueba de inicialización con elementos y prioridades válidos.
        heap = DWayHeap(elements=['A', 'B', 'C', 'D'], priorities=[0.1, -0.1, 1., -2.], branching_factor=2)
        self.assertEqual(4, len(heap))
        self.assertTrue(heap._validate())

    def test_heapify(self):
        # Prueba de la función heapify para distintos factores de ramificación.
        for b in BRANCHING_FACTORS_TO_TEST:
            # Se genera un tamaño aleatorio para el heap.
            size = 4 + random.randint(0, 20)
            # Se generan elementos (letras) y prioridades aleatorias.
            elements = [chr(i) for i in range(ord('A'), ord('A') + size)]
            priorities = [random.random() for _ in range(size)]
            heap = DWayHeap(elements=elements, priorities=priorities, branching_factor=b)
            self.assertEqual(size, len(heap))
            self.assertTrue(heap._validate())

    def test_clear(self):
        # Prueba del comportamiento en heaps vacíos y tras insertar elementos.
        for b in BRANCHING_FACTORS_TO_TEST:
            heap = DWayHeap(branching_factor=b)
            # Se verifica que llamar a peek en un heap vacío arroja RuntimeError.
            with self.assertRaises(RuntimeError) as context:
                heap.peek()
            self.assertTrue('Se llamó al método peek en un heap vacío.' in str(context.exception))

            # Inserta el primer elemento con una prioridad muy baja.
            heap.insert('First', -1e14)
            self.assertEqual('First', heap.peek())

            # Inserta varios elementos con distintas prioridades.
            heap.insert("b", 0)
            heap.insert("c", 0.99)
            heap.insert("second", 1)
            heap.insert("a", -11)
            self.assertEqual('second', heap.peek())

    def test_insert_top(self):
        # Prueba de la inserción y extracción (top) de elementos en el heap.
        for b in BRANCHING_FACTORS_TO_TEST:
            heap = DWayHeap(branching_factor=b)
            # Verifica que llamar a peek en un heap vacío arroja RuntimeError.
            with self.assertRaises(RuntimeError) as context:
                heap.peek()
            self.assertTrue('Se llamó al método peek en un heap vacío.' in str(context.exception))

            # Inserta el primer elemento y lo extrae utilizando top.
            heap.insert('First', -1e14)
            self.assertEqual('First', heap.top())
            self.assertTrue(heap.is_empty())

            # Inserta varios elementos y verifica que se extrae el de mayor prioridad.
            heap.insert("b", 0)
            heap.insert("c", 0.99)
            heap.insert("second", 1)
            heap.insert("a", -11)
            self.assertEqual('second', heap.top())
            self.assertEqual(3, len(heap))

            # Inserta 10 elementos adicionales con prioridades aleatorias.
            for i in range(10):
                heap.insert(str(i), random.random())

            # Extrae elementos hasta que el heap esté vacío, verificando la estructura en cada extracción.
            while not heap.is_empty():
                self.assertTrue(heap._validate())
                heap.top()



### 3. Perfilado y análisis de rendimiento

El tercer bloque de código se centra en analizar el rendimiento de la clase **DWayHeap** a través del módulo `cProfile` y `pstats`. Se buscan obtener métricas detalladas del tiempo de ejecución de métodos críticos (como `insert`, `_bubble_up`, `_push_down`, `top` y `_heapify`) para evaluar el comportamiento bajo distintas condiciones y factores de ramificación.

#### Estructura de la prueba de perfilado

El código define la clase `HeapProfile` (heredada de `unittest.TestCase`), en la que se incluyen tres pruebas de perfilado:

1. **Perfilado aislado de métodos (`test_profile_heap_methods_isolation`):**
   - Se realizan múltiples ejecuciones (5000 iteraciones) para cada factor de ramificación (rango de 2 a 20).
   - Se perfilan las operaciones de inserción (`insert`) y el método auxiliar `_bubble_up` en forma aislada.  
   - Además, se perfilan las extracciones (`top`) junto con el método `_push_down`.
   - Los resultados se escriben en un archivo CSV (`data/stats_heap.csv`) que recoge para cada método:  
     - Tiempo total de ejecución.
     - Tiempo acumulado.
     - Tiempo por llamada (calculado dividiendo el tiempo acumulado entre el número de llamadas).

2. **Perfilado de heapify (`test_profile_heapify`):**
   - Se genera un heap a partir de listas aleatorias de elementos y prioridades de tamaño variable (entre 1000 y 2000 elementos).
   - Se perfilan los métodos `_heapify` y `_push_down` durante la construcción del heap.
   - Los resultados se guardan en otro archivo CSV (`data/stats_heapify.csv`).

3. **Perfilado de interacción de métodos (`test_profile_heap_methods_interaction`):**
   - Se simulan operaciones mixtas en el heap: se inserta un elemento y, de manera aleatoria, se realizan extracciones (mientras el heap no esté vacío).
   - Esto permite evaluar el rendimiento en escenarios más cercanos al uso real.
   - Se recopilan estadísticas tanto para `insert` y `_bubble_up` como para `top` y `_push_down`.
   - Los datos se almacenan en el archivo `data/stats_heap_mixed.csv`.

#### Técnicas y herramientas utilizadas
- **cProfile:**  
  Se utiliza para capturar el rendimiento de cada llamada a métodos. Esto permite identificar cuellos de botella o posibles áreas de mejora en la implementación.
  
- **pstats:**  
  Se procesa la salida de `cProfile` para extraer métricas clave, filtrando la información relevante (por ejemplo, tiempos específicos de métodos del heap).
  
- **CSV para análisis:**  
  Los resultados se escriben en archivos CSV, lo que facilita su análisis posterior mediante herramientas de visualización o procesamiento de datos. Cada fila incluye detalles como el caso de prueba, el factor de ramificación, el nombre del método y las métricas de tiempo.

#### Consideraciones de diseño en el perfilado
- **Repetición y aleatoriedad:**  
  Realizar múltiples ejecuciones (5000 iteraciones) y utilizar datos aleatorios ayuda a obtener una medición estadísticamente relevante del rendimiento.
  
- **Variación del factor de ramificación:**  
  Al iterar sobre un amplio rango de valores (de 2 a 20), se puede evaluar cómo afecta el factor de ramificación a la eficiencia de las operaciones del heap.
  
- **Modularidad:**  
  La estructura del perfilado permite aislar pruebas específicas (por ejemplo, solo _heapify o solo inserción y extracción) lo que facilita la identificación de operaciones costosas.

In [ ]:
import cProfile
import pstats
import random
import unittest
from typing import List, Tuple

import os

if not os.path.exists('data'):
    os.makedirs('data', exist_ok=True)

# Se asume que la clase DWayHeap ya fue definida en la celda anterior o importada desde un módulo.

class HeapProfile(unittest.TestCase):
    BranchingFactors = range(2, 21)
    Runs = 5000
    OutputFileName = 'data/stats_heap.csv'
    OutputFileNameHeapify = 'data/stats_heapify.csv'
    OutputFileNameMixed = 'data/stats_heap_mixed.csv'

    @staticmethod
    def write_header(f) -> None:
        """
        Escribe la cabecera del archivo CSV para las estadísticas.
        """
        f.write('test_case,branching_factor,method_name,total_time,cumulative_time,per_call_time\n')

    @staticmethod
    def write_row(f, test_case: str, branching_factor: int, method_name: str, total_time: float,
                  cumulative_time: float, per_call_time: float) -> None:
        """
        Agrega una fila de datos al archivo CSV con las estadísticas.

        Parámetros:
            test_case: El caso de prueba.
            branching_factor: El factor de ramificación del heap.
            method_name: El nombre del método perfilado.
            total_time: Tiempo total del método.
            cumulative_time: Tiempo acumulado.
            per_call_time: Tiempo por llamada.
        """
        f.write(f'{test_case},{branching_factor},{method_name},{total_time},{cumulative_time},{per_call_time}\n')

    @staticmethod
    def get_running_times(st: pstats.Stats, method_name: str) -> List[Tuple[str, float, float, float]]:
        """
        Obtiene los tiempos de ejecución para métodos específicos del heap.

        Parámetro:
            st: Las estadísticas del perfilador.
            method_name: El nombre del método a filtrar.

        Retorna: Una lista de tuplas con el nombre del método, tiempo total, tiempo acumulado y tiempo por llamada.
        """
        ps = st.strip_dirs().stats

        # Filtrar métodos que contengan method_name en su descriptor
        def is_heap_method(k):
            return method_name in k[2]

        keys = list(filter(is_heap_method, ps.keys()))
        # Cada valor ps[key] contiene: (cc, nc, tt, ct, callers)
        return [(key[2], ps[key][2], ps[key][3], ps[key][3] / ps[key][1]) for key in keys]

    def test_profile_heap_methods_isolation(self) -> None:
        """
        Profilea de forma aislada los métodos 'insert' y '_bubble_up' del heap.
        Para cada factor de ramificación, se realizan varias ejecuciones y se registran las estadísticas.
        """
        with open(HeapProfile.OutputFileName, 'w') as f:
            HeapProfile.write_header(f)
            for b in HeapProfile.BranchingFactors:
                heap = DWayHeap(branching_factor=b)
                for _ in range(HeapProfile.Runs):
                    pro = cProfile.Profile()
                    pro.runcall(heap.insert, random.random(), random.random())
                    st = pstats.Stats(pro)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, 'insert'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, '_bubble_up'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)

                while not heap.is_empty():
                    pro = cProfile.Profile()
                    pro.runcall(heap.top)
                    st = pstats.Stats(pro)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, 'top'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)
                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, '_push_down'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)

    def test_profile_heapify(self) -> None:
        """
        Profilea el método _heapify y _push_down durante la creación del heap.
        Se generan listas aleatorias de elementos y prioridades, y se crea el heap para medir el rendimiento.
        """
        with open(HeapProfile.OutputFileNameHeapify, 'w') as f:
            HeapProfile.write_header(f)
            for b in HeapProfile.BranchingFactors:
                for _ in range(HeapProfile.Runs):
                    n = 1000 + random.randint(0, 1000)
                    elements = [random.random() for _ in range(n)]
                    pro = cProfile.Profile()
                    pro.runcall(DWayHeap, elements, elements, b)
                    st = pstats.Stats(pro)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, '_heapify'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, '_push_down'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)

    def test_profile_heap_methods_interaction(self) -> None:
        """
        Profilea la interacción de los métodos del heap al realizar operaciones mixtas.
        Se inserta un elemento y se, aleatoriamente, extraen elementos del heap, midiendo las estadísticas.
        """
        with open(HeapProfile.OutputFileNameMixed, 'w') as f:
            HeapProfile.write_header(f)
            for b in HeapProfile.BranchingFactors:
                heap = DWayHeap(branching_factor=b)
                for _ in range(HeapProfile.Runs):
                    pro = cProfile.Profile()
                    pro.runcall(heap.insert, random.random(), random.random())

                    while not heap.is_empty() and random.choice([True, False]):
                        pro.runcall(heap.top)

                    st = pstats.Stats(pro)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, 'insert'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, '_bubble_up'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, 'top'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)

                    for method_name, total_time, cumulative_time, per_call_time in \
                            HeapProfile.get_running_times(st, '_push_down'):
                        HeapProfile.write_row(f, 'heap', b, method_name, total_time, cumulative_time, per_call_time)


# Ejecuta la suite de pruebas de HeapProfile sin cerrar el kernel.
suite = unittest.TestLoader().loadTestsFromTestCase(HeapProfile)
unittest.TextTestRunner(verbosity=2).run(suite)

### Ejercicios

**Ejercicio 1**

- Demuestra que, para un heap d-ario con _n_ elementos, la altura del árbol es $ O(\log_d(n)) $.  
- Explica cómo varía la complejidad de las operaciones de inserción y extracción en función del factor de ramificación $ d $ y discute las implicaciones prácticas de elegir un valor alto o bajo para $ d $.

**Ejercicio 2**

- Explica la importancia de mantener las invariantes de un heap (por ejemplo, que cada nodo tenga la máxima prioridad en su subárbol) y detalla qué problemas pueden surgir si alguna de estas invariantes falla.  
- Propón una serie de escenarios (casos límite) que deberían probarse para garantizar que los métodos de _bubble up_ y _push down_ se comporten correctamente. Describe cómo diseñarías un conjunto de pruebas unitarias para detectar posibles errores en la implementación.

**Ejercicio 3**

- En el contexto de algoritmos como Dijkstra para encontrar el camino mínimo en grafos, diseña un enfoque que utilice un heap d-ario.  
- Describe cómo la elección de $ d $ puede afectar el rendimiento del algoritmo en grafos densos versus grafos dispersos.  
- Analiza el trade-off entre la rapidez de la operación de extracción (top) y la de inserción.

**Ejercicio 4**
- Imagina que tienes una lista de tareas, cada una con una prioridad y un tiempo de ejecución. Diseña un algoritmo basado en un heap d-ario para simular un scheduler que siempre ejecuta primero la tarea de mayor prioridad.  
- Describe cómo gestionarías la llegada de nuevas tareas (inserciones) y la finalización de tareas (extracciones) para mantener la eficiencia del scheduler.  
- Realiza un análisis teórico de la complejidad de tu solución y discute posibles cuellos de botella.

**Ejercicio 5**

- Supón que cuentas con dos implementaciones de un heap: una binaria ($d = 2$) y otra d-aria con $ d > 2 $. Diseña un experimento para comparar su rendimiento en operaciones de inserción y extracción.  
- **Puntos a considerar:**  
  - ¿Qué métricas utilizarías (tiempo total, tiempo acumulado, tiempo por llamada, etc.)?  
  - Explica cómo utilizar herramientas de perfilado (como cProfile y pstats) para obtener datos relevantes.  
  - Discute las expectativas de rendimiento y cómo interpretarías los resultados para decidir qué implementación es más adecuada en un contexto competitivo.

**Ejercicio 6**

- En un entorno de programación competitiva, el tiempo de ejecución y la robustez del código son cruciales. Describe cómo integrarías pruebas unitarias para una estructura de datos compleja (como el d-ary heap) durante el desarrollo sin comprometer el rendimiento en el entorno de producción.  
- Discute los desafíos que pueden surgir al diseñar casos de prueba para algoritmos con comportamientos no deterministas (por ejemplo, cuando se utiliza aleatoriedad en la generación de datos) y cómo solucionarlos.

**Ejercicio 7**

- Después de ejecutar un perfilador en tu implementación de heap d-ario, observas que ciertas funciones (como _bubble up_ o _push down_) consumen una cantidad desproporcionada de tiempo.  
- **Tarea:**  
  - Identifica qué indicadores y métricas serían clave para determinar el origen del cuello de botella.  
  - Propón posibles optimizaciones o reestructuraciones en la lógica de estas funciones para mejorar el rendimiento, justificando tu propuesta en términos de teoría de algoritmos y eficiencia en tiempo de ejecución.

In [ ]:
## Tu respuesta